<a href="https://colab.research.google.com/github/pwongsut/DNN_Lensing/blob/master/Lensing_Dnn_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.3f}'.format

#TODO: Change file name to match the uploaded file
filename = "SIS_data_n05.csv"

my_dataframe = pd.read_csv(filename)
my_dataframe

In [0]:
def preprocess_features(dataframe):
  """Prepares input features.

  Args:
    dataframe: A Pandas DataFrame expected to contain the data.
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = dataframe[list(dataframe)[20:30]]
  return selected_features

def preprocess_targets(dataframe, rmse=0.0):
  """Prepares target features (i.e., labels).

  Args:
    dataframe: A Pandas DataFrame expected to contain data.
    rmse: add uncertainty to the target using rmse (in 1e15 MSun)
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  # Scale the target to be in units of 1e15 MSun.
  output_targets["M200"] = (dataframe["M200"] / 1e15)
  if rmse > 0.0:
    mean = 0.0
    std  = rmse
    n    = output_targets.shape[0]
    output_targets["M200"] += np.random.normal(mean,std,n)
  return output_targets

In [0]:
# Shuffle data
my_dataframe = my_dataframe.iloc[np.random.permutation(len(my_dataframe))]

# Add rms error (in 1e15 Msun) to the learning target
rmse = 0.0

# Choose the first 6000 (out of 10000) examples for training.
training_examples = preprocess_features(my_dataframe.head(6000))
training_targets = preprocess_targets(my_dataframe.head(6000), rmse)

# Choose the 6001-8000 examples for validation.
validation_examples = preprocess_features(my_dataframe.iloc[6000:8000])
validation_targets = preprocess_targets(my_dataframe.iloc[6000:8000])

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

In [0]:
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a neural net regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                             
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

def train_nn_regression_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    continued=False,
    prev_regressor=None,
    prev_training_rmse=None,
    prev_validation_rmse=None):
  """Trains a neural network regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.
      
  Returns:
    A `DNNRegressor` object trained on the training data.
  """

  periods = 16
  steps_per_period = steps / periods
  
  # Create a DNNRegressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  if continued:
      dnn_regressor = prev_regressor
  else:
      dnn_regressor = tf.estimator.DNNRegressor(
          feature_columns=construct_feature_columns(training_examples),
          hidden_units=hidden_units,
          optimizer=my_optimizer,
      )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["M200"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["M200"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["M200"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  if continued:
    training_rmse = prev_training_rmse
    validation_rmse = prev_validation_rmse
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    dnn_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.3f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  print("Final RMSE (on training data):   %0.3f" % training_root_mean_squared_error)
  print("Final RMSE (on validation data): %0.3f" % validation_root_mean_squared_error)

  return dnn_regressor, training_rmse, validation_rmse

In [0]:
#---------------Start training the model---------------------
'''
default hyperparameters
    learning_rate=0.001,
    steps=10000,
    batch_size=150,
    hidden_units=[10, 6, 4]
'''
learning_rate=0.001
steps=10000
batch_size=150
hidden_units=[10, 6, 4]

dnn_regressor, training_rmse, validation_rmse = train_nn_regression_model(
    learning_rate=learning_rate,
    steps=steps,
    batch_size=batch_size,
    hidden_units=hidden_units,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

In [0]:
#Plot RMSE vs periods
plt.ylabel("RMSE")
plt.xlabel("Periods")
plt.title("Root Mean Squared Error vs. Periods with hyperparameters:\nlearning_rate = %g, batch_size = %d, hidden_units = " % (learning_rate,batch_size)+str(hidden_units))
plt.tight_layout()
plt.plot(training_rmse, label="training")
plt.plot(validation_rmse, label="validation")
#plt.ylim([0.05,0.3])
plt.legend()


In [0]:
#Continue training using previous dnn_regressor object
dnn_regressor, training_rmse, validation_rmse = train_nn_regression_model(
    learning_rate=learning_rate,
    steps=steps,
    batch_size=batch_size,
    hidden_units=hidden_units,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets,
    continued=True,
    prev_regressor=dnn_regressor,
    prev_training_rmse=training_rmse,
    prev_validation_rmse=validation_rmse)

In [0]:
#-------------------Testing the model on a test set-----------------------------

test_examples = preprocess_features(my_dataframe.tail(2000))
test_targets = preprocess_targets(my_dataframe.tail(2000))

predict_testing_input_fn = lambda: my_input_fn(test_examples, 
                                               test_targets["M200"], 
                                               num_epochs=1, 
                                               shuffle=False)

test_predictions = dnn_regressor.predict(input_fn=predict_testing_input_fn)
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE (on test data): %0.3f" % root_mean_squared_error)

In [0]:
#------------Testing the model on a test set from different file----------------

test_data = pd.read_csv("SIS_data_n05.csv", sep=",")
test_examples = preprocess_features(test_data.head(2000))
test_targets = preprocess_targets(test_data.head(2000))


predict_testing_input_fn = lambda: my_input_fn(test_examples, 
                                               test_targets["M200"], 
                                               num_epochs=1, 
                                               shuffle=False)

test_predictions = dnn_regressor.predict(input_fn=predict_testing_input_fn)
test_predictions = np.array([item['predictions'][0] for item in test_predictions])

root_mean_squared_error = math.sqrt(
    metrics.mean_squared_error(test_predictions, test_targets))

print("Final RMSE (on test data): %0.3f" % root_mean_squared_error)

In [0]:
#Show final rmse
print(training_rmse[-1])
print(validation_rmse[-1])

In [0]:
with open("savedata.txt",'w') as f:
  content = [training_rmse,validation_rmse]
  content = [ str(line)+"\t" for line in content ]
  f.writelines(content)
files.download('savedata.txt')